In [1]:
import matplotlib.pyplot as plt # For plotting
import numpy as np              # Linear algebra library
import pandas as pd







In [6]:
df = pd.read_csv('/content/drive/MyDrive/311/final_final_fr_fr.csv')

# Print unique values for each column
for column in df.columns:
    print(f"Unique values in {column}: {df[column].unique()}")

Unique values in id: [716549 715742 727333 606874 505318 605771 606929 609789 644623 626792
 714863 602805 602526 497421 389328 289931 620148 387155 516975 630825
 614043 521125 627780 492744 618747 607815 601609 602657 616449 501893
 507448 606556 724326 616945 630962 715814 502799 605278 388888 519548
 405521 607410 715795 625326 627220 634964 722500 602824 618355 611097
 634328 607160 728522 718570 714598 601950 612478 501416 629710 502610
 603476 600957 713423 722431 633234 148179 715845 632153 606552 738409
 612096 743537 632896 605207 617542 719554 627427 411315 631507 627798
 712093 627488 626493 601363 712898 603398 631012 601848 612131 408630
 632091 601122 602871 520828 514199 389697 714430 612567 744468 502722
 519681 631125 606119 627252 616568 603838 719735 608062 608704 520383
 603793 603933 605519 524803 629712 627505 526131 607065 500077 633350
 601945 629878 712766 603664 840827 604897 525422 602560 738379 622023
 604770 608318 626757 745297 627599 520037 632189 627636

In [45]:
def vectorized_one_hot(answers, attributes, attribute_to_index):
    indices_list = [
        [attribute_to_index[attr] for attr in ans if attr in attribute_to_index]
        for ans in answers
    ]

    # Create a 2D array filled with zeros
    one_hot_matrix = np.zeros((len(answers), len(attributes)), dtype=int)

    # Set appropriate positions to 1
    for i, indices in enumerate(indices_list):
        one_hot_matrix[i, indices] = 1

    return one_hot_matrix



q1 = """Q1: From a scale 1 to 5, how complex is it to make this food? (Where 1 is the most simple, and 5 is the most complex)"""
q2='Q2: How many ingredients would you expect this food item to contain?'
q3='Q3: In what setting would you expect this food to be served? Please check all that apply'
q4='Q4: How much would you expect to pay for one serving of this food item?'
q5='Q5: What movie do you think of when thinking of this food item?'
q6='Q6: What drink would you pair with this food item?'
q7='Q7: When you think about this food item, who does it remind you of?'
q8='Q8: How much hot sauce would you add to this food item?'
t='Label'

df[q2] = df[q2].replace("none", '0').astype(str)
df[q4] = df[q4].replace("none", '0').astype(str)

# one hot codings
q1_options = [1,2,3,4,5]
q3_options = ['none','Week day lunch','Week day dinner','Weekend lunch','Weekend dinner','At a party', 'Late night snack']
q7_options = ['Parents','Siblings','Friends', 'Teachers', 'Strangers' , 'none']
q8_options = ['I will have some of this food item with my hot sauce', 'A lot (hot)', 'A moderate amount (medium)', 'A little (mild)', 'none']

# one_hot = pd.get_dummies(df[q3], prefix="HotSauce")
# print(one_hot)

q3_attribute_to_index = {attr: idx for idx, attr in enumerate(q3_options)}
q3_ans = df[q3].astype(str).tolist()
q3_answers = [ans.split(",") for ans in q3_ans]
q3_hot = vectorized_one_hot(q3_answers, q3_options, q3_attribute_to_index)
print(q3_hot, '\n')

q7_attribute_to_index = {attr: idx for idx, attr in enumerate(q7_options)}
q7_ans = df[q7].astype(str).tolist()
q7_answers = [ans.split(",") for ans in q7_ans]
q7_hot = vectorized_one_hot(q7_answers, q7_options, q7_attribute_to_index)
print(q7_hot, '\n')

q8_attribute_to_index = {attr: idx for idx, attr in enumerate(q8_options)}
q8_ans = df[q8].astype(str).tolist()
q8_answers = [ans.split(",") for ans in q8_ans]
q8_hot = vectorized_one_hot(q8_answers, q8_options, q8_attribute_to_index)
print(q8_hot, '\n')

[[0 1 0 ... 0 1 1]
 [0 1 0 ... 0 1 1]
 [0 1 1 ... 1 0 0]
 ...
 [0 1 0 ... 0 1 0]
 [0 1 0 ... 0 0 1]
 [0 1 1 ... 1 0 1]] 

[[0 0 1 0 0 0]
 [0 0 1 1 1 0]
 [0 0 1 0 0 0]
 ...
 [1 0 0 0 0 0]
 [0 1 1 0 1 0]
 [1 1 1 0 1 0]] 

[[0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 ...
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 0 1]] 



In [48]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.preprocessing import (StandardScaler, LabelEncoder,
                                 MultiLabelBinarizer, OneHotEncoder,
                                 FunctionTransformer)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

# Define vectorized_one_hot function
def vectorized_one_hot(answers, options, attribute_to_index):
    num_samples = len(answers)
    num_attributes = len(options)
    one_hot_matrix = np.zeros((num_samples, num_attributes))

    for i, ans_list in enumerate(answers):
        for ans in ans_list:
            ans = ans.strip()
            if ans in attribute_to_index:
                one_hot_matrix[i, attribute_to_index[ans]] = 1
    return one_hot_matrix

# Question definitions
q1 = "Q1: From a scale 1 to 5, how complex is it to make this food? (Where 1 is the most simple, and 5 is the most complex)"
q2 = "Q2: How many ingredients would you expect this food item to contain?"
q3 = "Q3: In what setting would you expect this food to be served? Please check all that apply"
q4 = "Q4: How much would you expect to pay for one serving of this food item?"
q5 = "Q5: What movie do you think of when thinking of this food item?"
q6 = "Q6: What drink would you pair with this food item?"
q7 = "Q7: When you think about this food item, who does it remind you of?"
q8 = "Q8: How much hot sauce would you add to this food item?"
t = "Label"

# Data cleaning
df[q2] = df[q2].replace("none", '0').astype(str)
df[q4] = df[q4].replace("none", '0').astype(str)

# Define options for categorical columns
q1_options = [1,2,3,4,5]
q3_options = ['none','Week day lunch','Week day dinner','Weekend lunch','Weekend dinner','At a party', 'Late night snack']
q7_options = ['Parents','Siblings','Friends', 'Teachers', 'Strangers' , 'none']
q8_options = ['I will have some of this food item with my hot sauce', 'A lot (hot)', 'A moderate amount (medium)', 'A little (mild)', 'none']

# Create attribute to index mappings
q3_attribute_to_index = {attr: idx for idx, attr in enumerate(q3_options)}
q7_attribute_to_index = {attr: idx for idx, attr in enumerate(q7_options)}
q8_attribute_to_index = {attr: idx for idx, attr in enumerate(q8_options)}

# Manual feature engineering approach
df[q2] = pd.to_numeric(df[q2])
df[q1] = pd.to_numeric(df[q1])
df[q4] = pd.to_numeric(df[q4])

numerical_features = df[[q1, q2, q4]].values

# Multi-hot encoding
q3_hot = vectorized_one_hot([ans.split(",") for ans in df[q3].astype(str).tolist()], q3_options, q3_attribute_to_index)
q7_hot = vectorized_one_hot([ans.split(",") for ans in df[q7].astype(str).tolist()], q7_options, q7_attribute_to_index)
q8_hot = vectorized_one_hot([ans.split(",") for ans in df[q8].astype(str).tolist()], q8_options, q8_attribute_to_index)

# Frequency encoding
q5_encoded = df[q5].map(df[q5].value_counts(normalize=True)).to_frame()
q6_encoded = df[q6].map(df[q6].value_counts(normalize=True)).to_frame()

# Combine features
X = np.hstack([
    numerical_features,
    q3_hot,
    q7_hot,
    q8_hot,
    q5_encoded,
    q6_encoded
])

# Sparse version
X_sparse = sparse.hstack([
    sparse.csr_matrix(numerical_features),
    sparse.csr_matrix(q3_hot),
    sparse.csr_matrix(q7_hot),
    sparse.csr_matrix(q8_hot),
    sparse.csr_matrix(q5_encoded),
    sparse.csr_matrix(q6_encoded)
])

# Encode labels
le = LabelEncoder()
y = le.fit_transform(df["Label"])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Manual model
manual_model = LogisticRegression(
    multi_class="multinomial",
    solver="lbfgs",
    max_iter=1000
)
manual_model.fit(X_train, y_train)
print("Manual Feature Engineering Accuracy:", manual_model.score(X_test, y_test))

# Pipeline approach
def create_vectorized_one_hot(options, attribute_to_index):
    def transformer(X):
        return vectorized_one_hot([ans.split(",") for ans in X.astype(str)], options, attribute_to_index)
    return FunctionTransformer(transformer)

def create_frequency_encoder(col):
    def transformer(X):
        return X.map(col.value_counts(normalize=True)).values.reshape(-1, 1)
    return FunctionTransformer(transformer)

pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('num', StandardScaler(), [q1, q2, q4]),
        ('q3', create_vectorized_one_hot(q3_options, q3_attribute_to_index), [q3]),
        ('q7', create_vectorized_one_hot(q7_options, q7_attribute_to_index), [q7]),
        ('q8', create_vectorized_one_hot(q8_options, q8_attribute_to_index), [q8]),
        ('q5', create_frequency_encoder(df[q5]), [q5]),
        ('q6', create_frequency_encoder(df[q6]), [q6])
    ], remainder='drop')),
    ('classifier', LogisticRegression(
        multi_class='multinomial',
        solver='lbfgs',
        max_iter=1000,
        random_state=42
    ))
])

# Fit pipeline
pipeline.fit(df, y)

# Evaluate pipeline
print("Pipeline Accuracy:", pipeline.score(df, y))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Manual Feature Engineering Accuracy: 0.790273556231003


TypeError: the first argument must be callable

In [35]:




df[q2] = pd.to_numeric(df[q2])
df[q1] = pd.to_numeric(df[q1])
df[q4] = pd.to_numeric(df[q4])


numerical_features = df[[q1, q2, q4]].values  # or .to_numpy()

# Replace each movie/drink name with its frequency in the dataset
q5_encoded = df[q5].map(df[q5].value_counts(normalize=True)).to_frame()
q6_encoded = df[q6].map(df[q6].value_counts(normalize=True)).to_frame()

import numpy as np
from scipy import sparse

# Assuming you already have:
# q3_hot, q7_hot, q8_hot (from multi-hot encoding)
# numerical_features, q5_encoded, q6_encoded

X = np.hstack([
    numerical_features,
    q3_hot,          # Multi-hot for Q3
    q7_hot,          # Multi-hot for Q7
    q8_hot,          # Multi-hot for Q8
    q5_encoded,      # Encoded Q5 (movie)
    q6_encoded       # Encoded Q6 (drink)
])

# If using sparse matrices (for memory efficiency):
X_sparse = sparse.hstack([
    sparse.csr_matrix(numerical_features),
    sparse.csr_matrix(q3_hot),
    sparse.csr_matrix(q7_hot),
    sparse.csr_matrix(q8_hot),
    sparse.csr_matrix(q5_encoded),
    sparse.csr_matrix(q6_encoded)
])




from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df["Label"])  # e.g., Pizza → 0, Shawarma → 1, Sushi → 2

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model (multinomial for multi-class)
model = LogisticRegression(
    multi_class="multinomial",  # For >2 classes
    solver="lbfgs",            # Good for small datasets
    max_iter=1000              # Increase if convergence warning appears
)
model.fit(X_train, y_train)

# Evaluate
print("Accuracy:", model.score(X_test, y_test))

# First, import all required modules
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), [q1, q2, q4]),
        ("cat_q3", MultiLabelBinarizer(classes=q3_options), q3),
        ("cat_q7", MultiLabelBinarizer(classes=q7_options), q7),
        ("cat_q8", MultiLabelBinarizer(classes=q8_options), q8),
        ("text_q5", OneHotEncoder(handle_unknown="ignore"), [q5]),
        ("text_q6", OneHotEncoder(handle_unknown="ignore"), [q6])
    ]
)

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(multi_class="multinomial", solver="lbfgs"))
])

pipeline.fit(df, y)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.8115501519756839


TypeError: MultiLabelBinarizer.fit_transform() takes 2 positional arguments but 3 were given

In [36]:
# First ensure all required imports are at the top
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.preprocessing import (StandardScaler, LabelEncoder,
                                 MultiLabelBinarizer, OneHotEncoder)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import FunctionTransformer

# Your existing data preparation code
df[q2] = pd.to_numeric(df[q2])
df[q1] = pd.to_numeric(df[q1])
df[q4] = pd.to_numeric(df[q4])

numerical_features = df[[q1, q2, q4]].values

# Frequency encoding for Q5 and Q6
q5_encoded = df[q5].map(df[q5].value_counts(normalize=True)).to_frame()
q6_encoded = df[q6].map(df[q6].value_counts(normalize=True)).to_frame()

# Your multi-hot encoding (assuming vectorized_one_hot is defined)
q3_hot = vectorized_one_hot([ans.split(",") for ans in df[q3].tolist()], q3_options, q3_attribute_to_index)
q7_hot = vectorized_one_hot([ans.split(",") for ans in df[q7].tolist()], q7_options, q7_attribute_to_index)
q8_hot = vectorized_one_hot([ans.split(",") for ans in df[q8].tolist()], q8_options, q8_attribute_to_index)

# Combine features
X = np.hstack([
    numerical_features,
    q3_hot,
    q7_hot,
    q8_hot,
    q5_encoded,
    q6_encoded
])

# Sparse version - FIXED PARENTHESIS HERE
X_sparse = sparse.hstack([
    sparse.csr_matrix(numerical_features),
    sparse.csr_matrix(q3_hot),
    sparse.csr_matrix(q7_hot),
    sparse.csr_matrix(q8_hot),
    sparse.csr_matrix(q5_encoded),
    sparse.csr_matrix(q6_encoded)
])

# Encode labels
le = LabelEncoder()
y = le.fit_transform(df["Label"])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Your existing model
manual_model = LogisticRegression(
    multi_class="multinomial",
    solver="lbfgs",
    max_iter=1000
)
manual_model.fit(X_train, y_train)
print("Manual Feature Engineering Accuracy:", manual_model.score(X_test, y_test))

# Pipeline version (alternative approach)
# Define a custom wrapper for your vectorized_one_hot function
class VectorizedOneHotEncoder:
    def __init__(self, options, attribute_to_index):
        self.options = options
        self.attribute_to_index = attribute_to_index

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        answers = [ans.split(",") for ans in X]
        return vectorized_one_hot(answers, self.options, self.attribute_to_index)


# Define pipeline
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('num', StandardScaler(), [q1, q2, q4]),
        ('q3', VectorizedOneHotEncoder(q3_options, q3_attribute_to_index), q3),
        ('q7', VectorizedOneHotEncoder(q7_options, q7_attribute_to_index), q7),
        ('q8', VectorizedOneHotEncoder(q8_options, q8_attribute_to_index), q8),
        ('q5', Pipeline([
            ('freq_enc', FunctionTransformer(lambda x: x.map(x.value_counts(normalize=True)))),
            ('reshape', FunctionTransformer(lambda x: x.values.reshape(-1, 1)))
        ])),
        ('q6', Pipeline([
            ('freq_enc', FunctionTransformer(lambda x: x.map(x.value_counts(normalize=True)))),
            ('reshape', FunctionTransformer(lambda x: x.values.reshape(-1, 1)))
        ]))
    ])),
    ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000))
])

# Fit pipeline
pipeline.fit(df, y)

# Evaluate pipeline
print("Pipeline Accuracy:", pipeline.score(df, y))

Manual Feature Engineering Accuracy: 0.790273556231003


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


ValueError: not enough values to unpack (expected 3, got 2)

In [51]:


import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.preprocessing import (StandardScaler, LabelEncoder,
                                 FunctionTransformer)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Your vectorized_one_hot function
def vectorized_one_hot(answers, options, attribute_to_index):
    num_samples = len(answers)
    num_attributes = len(options)
    one_hot_matrix = np.zeros((num_samples, num_attributes))

    for i, ans_list in enumerate(answers):
        for ans in ans_list:
            ans = ans.strip()
            if ans in attribute_to_index:
                one_hot_matrix[i, attribute_to_index[ans]] = 1
    return one_hot_matrix

# Define question columns
questions = [q1, q2, q3, q4, q5, q6, q7, q8] = [
    "Q1: From a scale 1 to 5, how complex is it to make this food? (Where 1 is the most simple, and 5 is the most complex)",
    "Q2: How many ingredients would you expect this food item to contain?",
    "Q3: In what setting would you expect this food to be served? Please check all that apply",
    "Q4: How much would you expect to pay for one serving of this food item?",
    "Q5: What movie do you think of when thinking of this food item?",
    "Q6: What drink would you pair with this food item?",
    "Q7: When you think about this food item, who does it remind you of?",
    "Q8: How much hot sauce would you add to this food item?"
]
t = 'Label'


# Data cleaning
df[q2] = df[q2].replace("none", '0').astype(float)
df[q4] = df[q4].replace("none", '0').astype(float)
df[q1] = pd.to_numeric(df[q1])

# Define options for categorical columns
q3_options = ['none','Week day lunch','Week day dinner','Weekend lunch',
             'Weekend dinner','At a party', 'Late night snack']
q7_options = ['Parents','Siblings','Friends', 'Teachers', 'Strangers', 'none']
q8_options = ['I will have some of this food item with my hot sauce',
             'A lot (hot)', 'A moderate amount (medium)', 'A little (mild)', 'none']

# Create attribute to index mappings
q3_attribute_to_index = {attr: idx for idx, attr in enumerate(q3_options)}
q7_attribute_to_index = {attr: idx for idx, attr in enumerate(q7_options)}
q8_attribute_to_index = {attr: idx for idx, attr in enumerate(q8_options)}

# Manual feature engineering approach
numerical_features = df[[q1, q2, q4]].values

# Convert DataFrame columns to lists before splitting
q3_answers = [ans.split(",") for ans in df[q3].astype(str).tolist()]
q7_answers = [ans.split(",") for ans in df[q7].astype(str).tolist()]
q8_answers = [ans.split(",") for ans in df[q8].astype(str).tolist()]

q3_hot = vectorized_one_hot(q3_answers, q3_options, q3_attribute_to_index)
q7_hot = vectorized_one_hot(q7_answers, q7_options, q7_attribute_to_index)
q8_hot = vectorized_one_hot(q8_answers, q8_options, q8_attribute_to_index)

# Frequency encoding
q5_encoded = df[q5].map(df[q5].value_counts(normalize=True)).values.reshape(-1, 1)
q6_encoded = df[q6].map(df[q6].value_counts(normalize=True)).values.reshape(-1, 1)

# Combine features
X = np.hstack([
    numerical_features,
    q3_hot,
    q7_hot,
    q8_hot,
    q5_encoded,
    q6_encoded
])

# Encode labels
le = LabelEncoder()
y = le.fit_transform(df[t])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Manual model
manual_model = LogisticRegression(
    multi_class="multinomial",
    solver="lbfgs",
    max_iter=1000
)
manual_model.fit(X_train, y_train)
print("Manual Accuracy:", manual_model.score(X_test, y_test))

# Pipeline approach
def create_vectorized_one_hot(options, attribute_to_index):
    def transformer(X):
        X = pd.DataFrame(X).iloc[:, 0]  # Convert input to Series
        answers = [ans.split(",") for ans in X.astype(str)]
        return vectorized_one_hot(answers, options, attribute_to_index)
    return FunctionTransformer(transformer)

def create_frequency_encoder(col):
    def transformer(X):
        X = pd.DataFrame(X).iloc[:, 0]  # Convert input to Series
        return X.map(col.value_counts(normalize=True)).values.reshape(-1, 1)
    return FunctionTransformer(transformer)

pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('num', StandardScaler(), [q1, q2, q4]),
        ('q3', create_vectorized_one_hot(q3_options, q3_attribute_to_index), [q3]),
        ('q7', create_vectorized_one_hot(q7_options, q7_attribute_to_index), [q7]),
        ('q8', create_vectorized_one_hot(q8_options, q8_attribute_to_index), [q8]),
        ('q5', create_frequency_encoder(df[q5]), [q5]),
        ('q6', create_frequency_encoder(df[q6]), [q6])
    ], remainder='drop')),
    ('classifier', LogisticRegression(
        multi_class='multinomial',
        solver='lbfgs',
        max_iter=1000,
        random_state=42
    ))
])

# Fit pipeline
pipeline.fit(df[questions], y)

# Evaluate pipeline
print("Pipeline Accuracy:", pipeline.score(df[questions], y))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Manual Accuracy: 0.790273556231003
Pipeline Accuracy: 0.8071776155717761


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
